In [0]:
catalog = "workspace"


source_object = "silver_bookings"

source_schema = "silver"

cdc_column = "modified_date"

backdated_refresh = ""

fact_table = f"{catalog}.{source_schema}.{source_object}"

target_schema = "gold"

target_object = "FactBookings"

fact_key_cols = ["DimPassengersKey","DimFlightsKey","DimAirportsKey","booking_date"]


In [0]:
dimensions = [

    {

        "table": f"{catalog}.{target_schema}.DIMPassengers",
        "alias":"DIMPassenger",
        "join_keys": [("passenger_id","passenger_id")] #[fact,dim' ids]
    },
    {

        "table": f"{catalog}.{target_schema}.DIMFlights",
        "alias":"DIMFlight",
        "join_keys":[("flight_id","flight_id")] #[fact,dim' ids]
    },
    {

        "table": f"{catalog}.{target_schema}.DIMAirports",
        "alias":"DIMAirport",
        "join_keys":[("airport_id","airport_id")] #[fact,dim' ids]
    }



]

fact_columns = ["amount","booking_date","modified_date"]

Last load

In [0]:
if len(backdated_refresh) == 0:
    if spark.catalog.tableExists(f"workspace.{target_schema}.{target_object}"):
        last_load = spark.sql(f"Select max({cdc_col}) from workspace.{target_schema}.{target_object}").collect[0][0]
    else:
        last_load = "1900-01-01 00:00:00"
else: 
    last_load = backdated_refresh 

Dynamic Fact Table

In [0]:
def generate_fact_query_incremental(fact_table,dimensions,fact_columns,cdc_column):
    fact_alias = "f"

    #Base columns to select
    select_cols = [f"{fact_alias}.{col}" for col in fact_columns]

    #Join columns
    join_cols = []
    for dim in dimensions:
        table_full = dim["table"]
        alias = dim["alias"]
        table_name = table_full.split(".")[-1]
        surrogate_key = f"{alias}.{table_name}Key"
        select_cols.append(surrogate_key)

        on_conditions = [
            f"{fact_alias}.{fk} = {alias}.{dk}"
            for dk,fk in dim["join_keys"]
        
        ]
        join_clause = f"LEFT JOIN {table_full} {alias} ON "+" AND ".join(on_conditions)
        join_cols.append(join_clause)
    
    select_clause = ", ".join(select_cols)
    joins = "\n".join(join_cols)

    where_clause = f"{fact_alias}.{cdc_column} >= DATE('{last_load}')"
   


    query = f"""

SELECT
    {select_clause}
FROM {fact_table} {fact_alias}
{joins}

WHERE {where_clause}
""".strip()

    return query

In [0]:
query = generate_fact_query_incremental(fact_table,dimensions,fact_columns,cdc_column)


In [0]:
print(query)

In [0]:
df_fact = spark.sql(query)

In [0]:
df_fact.display()

In [0]:
from delta.tables import DeltaTable



In [0]:
fact_key_cols_str = "AND" .join([f"src.{col} = trg.{col}" for col in fact_key_cols])


In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    dlt_obj = DeltaTable.forName(spark, f"{catalog}.{target_schema}.{target_object}")
    dlt_obj.alias("trg").merge(df_fact.alias("src"), fact_cols_str)\
    .whenMatchedUpdateAll(condition=f"src.{cdc_column} >= trg.{cdc_column}")\
    .whenNotMatchedInsertAll()\
    .execute()
else:
    df_fact.write.format("delta")\
    .mode("append")\
    .saveAsTable(f"{catalog}.{target_schema}.{target_object}")

In [0]:
%sql
SELECT * FROM workspace.gold.factbookings;